<a href="https://colab.research.google.com/github/Maria1007mart/sort_news_2/blob/main/byes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!wget http://setup.johnsnowlabs.com/kaggle.sh -O - | bash 

--2023-03-29 09:31:38--  http://setup.johnsnowlabs.com/kaggle.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://setup.johnsnowlabs.com/kaggle.sh [following]
--2023-03-29 09:31:38--  https://setup.johnsnowlabs.com/kaggle.sh
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/kaggle_setup.sh [following]
--2023-03-29 09:31:39--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/kaggle_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.13

In [ ]:
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.ml.feature import HashingTF, IDF, StringIndexer, IndexToString
from pyspark.ml.classification import NaiveBayes
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
import sparknlp

In [ ]:
spark = sparknlp.start()

In [12]:
main_df = spark.read.csv(
    'lenta-ru-news.csv',
    header=True,
    multiLine=True,
    escape="\"")
main_df.show()

+--------------------+--------------------+--------------------+----------+--------------+----------+
|                 url|               title|                text|     topic|          tags|      date|
+--------------------+--------------------+--------------------+----------+--------------+----------+
|https://lenta.ru/...|1914. Русские вой...|Бои у Сопоцкина и...|Библиотека|Первая мировая|1914/09/16|
|https://lenta.ru/...|1914. Праздновани...|Министерство наро...|Библиотека|Первая мировая|1914/09/16|
|https://lenta.ru/...|1914. Das ist Nes...|Штабс-капитан П. ...|Библиотека|Первая мировая|1914/09/17|
|https://lenta.ru/...|1914. Бульдог-гон...|Фотограф-корреспо...|Библиотека|Первая мировая|1914/09/17|
|https://lenta.ru/...|1914. Под Люблино...|Лица, приехавшие ...|Библиотека|Первая мировая|1914/09/18|
|https://lenta.ru/...|Космонавты сомнев...|Как стало известн...|    Россия|           Все|1999/08/31|
|https://lenta.ru/...|Взрыв в центре Мо...|В зале игровых ав...|    Россия|       

In [13]:
main_df.count()

15901

In [14]:
filtered_df = main_df \
    .na.drop(subset=["topic"]) \
    .select(["text", "topic"]) \
    .withColumn("text", F.regexp_replace(F.col("text"), "[\n\r]", " ")) \
    .withColumn("text", F.regexp_replace(F.col("text"), ".Rambler Title ", "")) \
    .withColumn("text", F.trim(F.col("text")))
filtered_df.show()

+--------------------+----------+
|                text|     topic|
+--------------------+----------+
|Бои у Сопоцкина и...|Библиотека|
|Министерство наро...|Библиотека|
|Штабс-капитан П. ...|Библиотека|
|Фотограф-корреспо...|Библиотека|
|Лица, приехавшие ...|Библиотека|
|Как стало известн...|    Россия|
|В зале игровых ав...|    Россия|
|Япония приняла ре...|    Россия|
|Британцы отмечают...|       Мир|
|В понедельник дир...|    Россия|
|С 1 сентября на в...|    Россия|
|Указом президента...|    Россия|
|Сегодня областной...|    Россия|
|Бывший шеф Службы...|    Россия|
|подземный толчок ...|       Мир|
|Сегодня утром в р...|    Россия|
|Намеченная на сег...|    Россия|
|На состоявшейся с...|    Россия|
|15 представителей...|       Мир|
|На юге Киргизии, ...|       Мир|
+--------------------+----------+
only showing top 20 rows



In [15]:
filtered_df.count()

17416

In [16]:
count_df = filtered_df\
    .groupBy("topic")\
    .count()\
    .orderBy(F.col("count").desc())
count_df.show()

+--------------+-----+
|         topic|count|
+--------------+-----+
|        Россия| 8596|
|           Мир| 3736|
|     Экономика| 2400|
|Интернет и СМИ| 1594|
|         Спорт|  845|
|      Культура|  557|
|      Из жизни|  197|
|    Библиотека|    5|
+--------------+-----+



In [17]:
count_df.count()

8

In [18]:
selected_rows = count_df\
    .select("topic")\
    .limit(5)\
    .collect()
selected_topics = [row.topic for row in selected_rows]
selected_topics

['Россия', 'Мир', 'Экономика', 'Интернет и СМИ', 'Спорт']

In [19]:
df = filtered_df\
    .filter(F.col("topic").isin(selected_topics))
df.show()

+--------------------+------+
|                text| topic|
+--------------------+------+
|Как стало известн...|Россия|
|В зале игровых ав...|Россия|
|Япония приняла ре...|Россия|
|Британцы отмечают...|   Мир|
|В понедельник дир...|Россия|
|С 1 сентября на в...|Россия|
|Указом президента...|Россия|
|Сегодня областной...|Россия|
|Бывший шеф Службы...|Россия|
|подземный толчок ...|   Мир|
|Сегодня утром в р...|Россия|
|Намеченная на сег...|Россия|
|На состоявшейся с...|Россия|
|15 представителей...|   Мир|
|На юге Киргизии, ...|   Мир|
|Россия крайне нег...|   Мир|
|По сведениям мисс...|   Мир|
|Соединенные Штаты...|   Мир|
|"Женщины России" ...|Россия|
|Скандал по поводу...|   Мир|
+--------------------+------+
only showing top 20 rows



In [20]:
df.count()

19699

In [21]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed = 1)

In [22]:
train_df.groupBy("topic")\
    .count()\
    .orderBy(F.col("count").desc())\
    .show()

+--------------+-----+
|         topic|count|
+--------------+-----+
|        Россия| 7880|
|           Мир| 3680|
|     Экономика| 2228|
|Интернет и СМИ| 1510|
|         Спорт|  793|
+--------------+-----+



In [23]:
test_df.groupBy("topic")\
    .count()\
    .orderBy(F.col("count").desc())\
    .show()

+--------------+-----+
|         topic|count|
+--------------+-----+
|        Россия| 2028|
|           Мир|  965|
|     Экономика|  568|
|Интернет и СМИ|  414|
|         Спорт|  216|
+--------------+-----+



In [24]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

In [25]:
sentence_detector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

In [26]:
tokenizer = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

In [27]:
stop_words_cleaner = StopWordsCleaner\
    .pretrained('stopwords_ru', 'ru')\
    .setInputCols(["token"]) \
    .setOutputCol("cleanTokens") \
    .setCaseSensitive(False)

stopwords_ru download started this may take some time.
Approximate size to download 2.9 KB
[OK!]


In [28]:
lemmatizer = LemmatizerModel\
    .pretrained("lemma", "ru") \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

lemma download started this may take some time.
Approximate size to download 1.3 MB
[OK!]


In [29]:
finisher = Finisher() \
    .setInputCols(["lemma"]) \
    .setOutputCols(["token_features"]) \
    .setOutputAsArray(True) \
    .setCleanAnnotations(False)

In [30]:
hashing_TF = HashingTF(
    inputCol="token_features",
    outputCol="raw_features")

In [31]:
idf = IDF(
    inputCol="raw_features",
    outputCol="features",
    minDocFreq=5)

In [32]:
topic_indexer = StringIndexer(
    inputCol="topic",
    outputCol="label")

In [33]:
bayes_classificator = NaiveBayes(
    smoothing=111)

In [34]:
topic_to_string_indexer = IndexToString(
    inputCol="label",
    outputCol="article_class")

In [35]:
pipeline = Pipeline(
    stages=[
        document_assembler,
        sentence_detector,
        tokenizer,
        stop_words_cleaner,
        lemmatizer,
        finisher,
        hashing_TF,
        idf,
        topic_indexer,
        bayes_classificator,
        topic_to_string_indexer])

In [36]:
%%time
classification_model = pipeline.fit(train_df)

CPU times: user 2 s, sys: 247 ms, total: 2.25 s
Wall time: 5min 43s


In [37]:
from sklearn.metrics import classification_report, accuracy_score

In [38]:
df_bayes = classification_model \
    .transform(test_df) \
    .select("topic", "label", "prediction", "text")
df_bayes_pandas = df_bayes.toPandas()

In [39]:
labels_df = df_bayes\
    .select("label", "topic")\
    .distinct()\
    .orderBy("label")
labels_df.show()

+-----+--------------+
|label|         topic|
+-----+--------------+
|  0.0|        Россия|
|  1.0|           Мир|
|  2.0|     Экономика|
|  3.0|Интернет и СМИ|
|  4.0|         Спорт|
+-----+--------------+



In [40]:
labels_raw = labels_df.collect()
labels = [row.topic for row in labels_raw]
labels

['Россия', 'Мир', 'Экономика', 'Интернет и СМИ', 'Спорт']

In [41]:
print(classification_report(
    df_bayes_pandas.label, df_bayes_pandas.prediction, target_names=labels))

                precision    recall  f1-score   support

        Россия       0.58      0.99      0.73      5181
           Мир       0.93      0.43      0.59      3445
     Экономика       0.96      0.41      0.57      1453
Интернет и СМИ       0.94      0.21      0.35       917
         Спорт       1.00      0.68      0.81       749

      accuracy                           0.67     11745
     macro avg       0.88      0.54      0.61     11745
  weighted avg       0.79      0.67      0.65     11745



In [42]:
print(accuracy_score(
    df_bayes_pandas.label, df_bayes_pandas.prediction))

0.6738186462324394
